In [1]:
from poly2pascal.annotations import XMLAnnotator
from tqdm.notebook import tqdm
from tqdm.notebook import trange
from time import sleep
import pandas as pd
import numpy as np

import os
import random

In [29]:
bbox_file = "/Users/77369/PycharmProjects/641/data/images_bboxes_withoutNaN.csv"
bbox_df = pd.read_csv(bbox_file)

valid_id = bbox_df['image_id'].unique()
valid_id = [f.split('.')[0] for f in valid_id]

print(len(valid_id))
# bbox_df.dropna(how='any',inplace=True)


# bbox_df.to_csv("./data/images_bboxes_withoutNaN.csv")


6479


In [19]:
# create annotator

# csv_file_path="/Users/77369/PycharmProjects/641/data/images_bboxes_withoutNaN.csv", 
xmla = XMLAnnotator(
    images_path="/Users/77369/mmdetection/data/VOCdevkit/VOC2007/JPEGImages", 
    csv_file_path="/Users/77369/PycharmProjects/641/data/images_bboxes_withoutNaN.csv", 
    image_name_col="image_id",
    image_label_col="worm_type", 
    xml_output_path="/Users/77369/mmdetection/data/VOCdevkit/VOC2007/Annotations",
    geometry_col="geometry",
    xml_end_content="\n</annotation>"
)

# create xml annotation files in Pascal VOC format
xmla.get_all_xml_annotations(img_format=".jpg")

In [3]:
trainval_percent = 0.5
train_percent = 0.5
xmlfilepath = '/Users/77369/mmdetection/data/VOCdevkit/VOC2007/Annotations'
txtsavepath = '/Users/77369/mmdetection/data/VOCdevkit/VOC2007/ImageSets/Main'
total_xml = os.listdir(xmlfilepath)

num = len(total_xml)
list = range(num)
tv = int(num * trainval_percent)
tr = int(tv * train_percent)
trainval = random.sample(list, tv)
train = random.sample(trainval, tr)

ftrainval = open(txtsavepath + '/trainval.txt', 'w')
ftest = open(txtsavepath + '/test.txt', 'w')
ftrain = open(txtsavepath + '/train.txt', 'w')
fval = open(txtsavepath + '/val.txt', 'w')

for i in list:
    name = total_xml[i][:-4] + '\n'
    if i in trainval:
        ftrainval.write(name)
        if i in train:
            ftrain.write(name)
        else:
            fval.write(name)
    else:
        ftest.write(name)

ftrainval.close()
ftrain.close()
fval.close()
ftest.close()


In [39]:
# trainval_percent = 0.5
train_percent = 0.5
bbox_file = "/Users/77369/PycharmProjects/641/data/images_bboxes_withoutNaN.csv"
xmlfilepath = '/Users/77369/mmdetection/data/VOCdevkit/VOC2007/Annotations'
trainpath ='./data/Train.csv' 
testpath = './data/Test.csv'
txtsavepath = '/Users/77369/mmdetection/data/VOCdevkit/VOC2007/ImageSets/Main/'
total_xml = os.listdir(xmlfilepath)
total_xml = [f.split('.')[0] for f in total_xml]

bbox_df = pd.read_csv(bbox_file)
valid_id = bbox_df['image_id'].unique()
valid_id = [f.split('.')[0] for f in valid_id]

train_data = pd.read_csv(trainpath)
test_data = pd.read_csv(testpath)

train_data['image_id_worm'] = train_data['image_id_worm'].apply(lambda x: x.split('.')[0])
train_data = train_data[train_data['image_id_worm'].isin(valid_id)]
train_data = train_data[train_data['image_id_worm'].isin(total_xml)]
train_data.drop_duplicates(subset=['image_id_worm'], inplace=True)
train_data.reset_index(drop=True, inplace=True)


data_save = train_data.loc[:1999, 'image_id_worm']

num = data_save.shape[0]
list = range(num)
tr = int(num * train_percent)
train = random.sample(list, tr)
val = [item for item in list if item not in train]

train_data_save = data_save[train]
val_data_save = data_save[val]

data_save.to_csv(txtsavepath+'trainval.txt', sep='\n', index=False, header=False)
train_data_save.to_csv(txtsavepath+'train.txt', sep='\n', index=False, header=False)
val_data_save.to_csv(txtsavepath+'test.txt', sep='\n', index=False, header=False)

In [ ]:
for pic in test_pic_df.values.tolist()[500:]:
# for pic in show_data:
    img = pic_file_root + pic[0] + '.jpg'  # or img = mmcv.imread(img), which will only load it once
    result = inference_detector(model, img)
    pre_pwb = np.sum(result[0][:, -1] >= threshold)
    pre_awb = np.sum(result[1][:, -1] >= threshold)
    true_info = train_pic_df[train_pic_df['image_id_worm'] == pic[0] + '.jpg']
    if 'pbw' in true_info['worm_type'].values:
        y_pbw = true_info[true_info['worm_type'] == 'pbw']['number_of_worms'].values[0]
    else:
        y_pbw = 0
    if 'abw' in true_info['worm_type'].values:
        y_abw = true_info[true_info['worm_type'] == 'abw']['number_of_worms'].values[0]
    else:
        y_abw = 0
    mae_temp = abs(pre_pwb - y_pbw) +abs(pre_awb - y_abw)
    mae += mae_temp

    pre_awb_total.append(pre_awb)
    pre_pwb_total.append(pre_awb)
    y_awb_total.append(y_abw)
    y_pwb_total.append(y_pbw)
    mae_total.append(mae_temp)